In [1]:
# Importing required libraries
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
print ("TensorFlow version: " + tf.__version__)

TensorFlow version: 2.1.0


In [2]:
# Importing the Model
model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_tensor=tf.keras.Input(shape=(224,224,3)))

In [3]:
# Freezing layers for Fine Tuning
for layer in model.layers:
	layer.trainable = False

In [4]:
# Manipulating layers for custom classes
x = model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.5, name='drop0')(x)
x = tf.keras.layers.Flatten(name='flatten')(x)
x = tf.keras.layers.Dense(1024, activation=tf.nn.relu, name="fc1")(x)
x = tf.keras.layers.Dropout(0.5, name="drop1")(x)
x = tf.keras.layers.Dense(1024, activation=tf.nn.relu, name='fc2')(x)
x = tf.keras.layers.Dropout(0.5, name='drop2')(x)
x = tf.keras.layers.Dense(5, activation=tf.nn.softmax, name='predictions')(x)
custom_model = tf.keras.Model(inputs=model.input, outputs=x)

In [5]:
# # Visualizing Model Summary
# custom_model.summary()

In [6]:
# Visualizing Model Architecture
# tf.keras.utils.plot_model(custom_model,'model.png')

In [7]:
# Preparing Training, Validation and Testing Data
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.25)
# , zoom_range=0.2, shear_range=0.2, rotation_range=45, horizontal_flip=True, width_shift_range=.15, height_shift_range=.15)
train_generator = train_datagen.flow_from_directory('/home/bhaven/Downloads/dataset/resized_hundred', batch_size=64, target_size=(224, 224), class_mode='sparse', subset='training', shuffle=True)
validation_generator = train_datagen.flow_from_directory('/home/bhaven/Downloads/dataset/resized_hundred', batch_size=64, target_size=(224, 224), class_mode='sparse', subset='validation', shuffle=True)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('/home/bhaven/Downloads/dataset/resized_uneven_test', target_size=(224,224), batch_size=50, class_mode ='sparse', shuffle=True)

Found 96786 images belonging to 5 classes.
Found 32259 images belonging to 5 classes.
Found 53576 images belonging to 5 classes.


In [8]:
# Compiling the Model with RMSprop
custom_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(name='sparse_categorical_crossentropy'), optimizer=tf.keras.optimizers.RMSprop(name='RMSprop'), metrics=['accuracy'])

In [9]:
custom_model.fit(train_generator, epochs=5, validation_data=validation_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1513 steps, validate for 505 steps
Epoch 1/5
1513/1513 [==============================] - 652s 431ms/step - loss: 1.2842 - accuracy: 0.4664 - val_loss: 1.2244 - val_accuracy: 0.4822
Epoch 2/5
1513/1513 [==============================] - 641s 424ms/step - loss: 1.1903 - accuracy: 0.5115 - val_loss: 1.3418 - val_accuracy: 0.4379
Epoch 3/5
1513/1513 [==============================] - 642s 425ms/step - loss: 1.1715 - accuracy: 0.5222 - val_loss: 1.3475 - val_accuracy: 0.4634
Epoch 4/5
1513/1513 [==============================] - 645s 427ms/step - loss: 1.1570 - accuracy: 0.5298 - val_loss: 1.3803 - val_accuracy: 0.4495
Epoch 5/5
1513/1513 [==============================] - 645s 426ms/step - loss: 1.1517 - accuracy: 0.5361 - val_loss: 1.4546 - val_accuracy: 0.4162


In [10]:
## Visualizing layers to see which layers to freeze
# for i, layer in enumerate(model.layers):
#     print(i, layer.name)

In [11]:
# Freezing first 249 layers and unfreezing the rest
for layer in custom_model.layers[:249]:
    layer.trainable = False
for layer in custom_model.layers[249:]:
    layer.trainable = True

In [12]:
# Reompiling the Model with Adam
custom_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(name='sparse_categorical_crossentropy'), optimizer=tf.keras.optimizers.SGD(learning_rate=.0001, name='SGD', momentum=0.9), metrics=['accuracy'])

In [13]:
## Saving Model Checkpoints
# checkpoint_path = "training_1/cp-{epoch:04d}.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# cp_callback = tf.keras.callbacks.ModelCheckpoint(
#     checkpoint_path, monitor='val_accuracy', verbose=0, save_best_only=False,
#     save_weights_only=False, mode='auto', save_freq='epoch')

In [ ]:
# Training the Model
history = custom_model.fit(train_generator, epochs=5, validation_data=validation_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1513 steps, validate for 505 steps
Epoch 1/5
1513/1513 [==============================] - 736s 486ms/step - loss: 1.0487 - accuracy: 0.5780 - val_loss: 1.2726 - val_accuracy: 0.4930
Epoch 2/5
1513/1513 [==============================] - 726s 480ms/step - loss: 0.9470 - accuracy: 0.6179 - val_loss: 1.2761 - val_accuracy: 0.4899
Epoch 3/5
 127/1513 [=>............................] - ETA: 8:51 - loss: 0.8775 - accuracy: 0.6425

In [ ]:
# Calculating Model Test Score
score = custom_model.evaluate(test_generator, verbose=1)
print('Test accuracy:', 100*score[1])
print('\nTest loss:', score[0])

In [ ]:
# Visualizing Training and Test attributes
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.style.use('ggplot')
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.plot(train_loss,label='Training Loss')
plt.plot(val_loss,label='Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.title('train_loss and val_loss')
plt.legend(loc='upper right')

plt.subplot(1,2,2)
plt.plot(train_acc,label='Training Accuracy')
plt.plot(val_acc,label='Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.title('train_acc and val_acc')
plt.legend(loc='lower right')

In [ ]:
# Saving Model
custom_model.save('/home/bhaven/Downloads/Models/diabetic.h5')

# architecture to JSON, weights to HDF5
custom_model.save_weights('/home/bhaven/Downloads/Models/weights.h5')
with open('/home/bhaven/Downloads/Models/architecture.json', 'w') as f:
        f.write(custom_model.to_json())

In [ ]:
# Loading Saved Model
# architecture and weights from HDF5
model = tf.keras.models.load_model('/home/bhaven/Downloads/Models/diabetic.h5')
# architecture from JSON, weights from HDF5
with open('/home/bhaven/Downloads/Models/architecture.json') as f:
    model = tf.keras.models.model_from_json(f.read())
model.load_weights('/home/bhaven/Downloads/Models/weights.h5')

In [ ]:
# Getting test batch and label batch
for image_batch, label_batch in test_generator:
    print("Image batch shape: ", image_batch.shape)
    print("Label batch shape: ", label_batch.shape)
    break
# CLASS_NAMES = np.array(['0','1','2','3','4'])
label_batch.astype(int)

In [ ]:
# Predicting test batch
prediction = model.predict(image_batch)

In [ ]:
# Visualizing Predictions
plt.figure(figsize=(25,50))
plt.style.use('ggplot')
for n in range(test_generator.batch_size):
    ax = plt.subplot(17,3,n+1)
    plt.imshow(image_batch[n])
    plt.title("Lvl {}=>Lvl 0={:.0f}%, Lvl 1={:.0f}%, Lvl 2={:.0f}%, Lvl 3={:.0f}%, Lvl 4={:.0f}%".format(label_batch[n].astype(int),100*prediction[n,0],100*prediction[n,1],100*prediction[n,2],100*prediction[n,3],100*prediction[n,4]))
    plt.axis('off')
# plt.savefig('pred.jpeg')